In [ ]:
import pandas as pd
import portfolio
from stats import *
from visualisation import plot_portfolio_value, plot_trade_points

In [ ]:
# Read in the data
ticker = "AAPL"
data_path = f"../data/{ticker}_processed_hourly_data.csv"
data = pd.read_csv(data_path, index_col='Datetime', parse_dates=True)

In [ ]:
def calculate_macd(df, fast_period=10, slow_period= 25, signal_period=40):
    df['Fast EMA'] = df['Close'].ewm(span=fast_period, adjust=False).mean()
    df['Slow EMA'] = df['Close'].ewm(span=slow_period, adjust=False).mean()
    df['MACD'] = df['Fast EMA'] - df['Slow EMA']
    df['Signal Line'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
    return df

In [ ]:
data = calculate_macd(data)

In [ ]:
# Initialise the portfolio
portfolio = portfolio.Portfolio(initial_cash=100000)
stop_loss = 0.02
take_profit = 0.1

# Prepare df to track data
data["Portfolio Value"] = None
data["Actions"] = None

# Time period (for days set it to 252 and for hourly set to 1638)
time_period = 1638

In [ ]:
for index, row in data.iterrows():
    current_price = row['Close']
    data.at[index, "Portfolio Value"] = portfolio.get_value(current_price)

    if portfolio.holdings and portfolio.last_buy_price:
        if portfolio.last_buy_price <= (current_price * (1 - stop_loss)):
            portfolio.sell(current_price, 250)
            data.at[index, "Actions"] = -1
        elif portfolio.last_buy_price >= (current_price * (1 + take_profit)):
            portfolio.buy(current_price, 250)
            data.at[index, "Actions"] = 1

    # Buy: MACD crosses above Signal Line
    if row["MACD"] > row["Signal Line"] and not portfolio.holdings:
        portfolio.buy(current_price, 250)
        data.at[index, "Actions"] = 1
    # Sell: MACD crosses below Signal Line
    elif row["MACD"] <= row["Signal Line"] and portfolio.holdings:
        portfolio.sell(current_price, 250)
        data.at[index, "Actions"] = -1

print(f"Portfolio Final Value: {portfolio.get_value(data.iloc[-1]['Close'])}")
data['Returns'] = data['Portfolio Value'].pct_change().dropna()

In [ ]:
years = ((data.index[-1] - data.index[0]).days / 365)

cagr = calculate_compound_annual_growth_rate(data.iloc[0]['Portfolio Value'], data.iloc[-1]['Portfolio Value'], years)
sharpe_ratio = calculate_sharpe_ratio(data['Returns'], periods_per_year=time_period)
max_drawdown = calculate_max_drawdown(data['Portfolio Value'].dropna().tolist())
volatility = calculate_volatility(data['Returns'], periods_per_year=time_period)

In [ ]:
print(f"Compound Annual Growth Rate: {cagr}")
print(f"Sharpe Ratio: {sharpe_ratio}")
print(f"max_drawdown: {max_drawdown}")
print(f"volatility: {volatility}")

In [ ]:
plot_portfolio_value(data, "Moving Average Convergence Divergence")

In [ ]:
plot_trade_points(data.iloc[-500:], "Moving Average Convergence Divergence")